In [13]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for httpx==0.13.3 from https://files.pythonhosted.org/packages/54/b4/698b284c6aed4d7c2b4fe3ba5df1fcf6093612423797e76fbb24890dd22f/httpx-0.13.3-py3-none-any.whl.metadata
  Obtaining dependency information for hstspreload from https://files.pythonhosted.org/packages/25/2a/e9ca0cdf8bd33d61d8d237b19e62d5c2f2666e482ae3fd45e0a80b8dcfd0/hstspreload-2024.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for chardet==3.* from https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl.metadata
  Obtaining dependency information for idna==2.* from https://files.pythonhosted.org/packages/a2/38/928ddce2273eaa564f6f50de919327bf3a00f091b5baba8dfa9460f3a8a8/idna-2.10-py2.py3-none-any.whl.metadata
  Obtaining dependency information for rfc3986<2,>=1.3 from https://files.p

In [5]:
import os

In [9]:
os.getcwd()

'D:\\study\\Python_JupyterNotebook\\RecommendGame\\크롤링'

In [2]:
def webonload():
    driver = webdriver.Chrome()
    driver.maximize_window() # 브라우저 화면 최대화
    driver.get('https://store.steampowered.com/search/?filter=topsellers&os=win&lang=ko')
    time.sleep(1)
    SCROLL_PAUSE_SEC = 1
    translator = Translator() # 번역기 생성
    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    return driver

In [3]:
def rep_char(input_string):
    special_characters = ['\\', '/', ':', '*', '?', '"', '<', '>', '|']
    result_string = input_string

    for char in special_characters:
        result_string = result_string.replace(char, '')

    return result_string

In [4]:
def crawling(driver):
    SCROLL_PAUSE_SEC = 1
    translator = Translator() # 번역기 생성
    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")
    tbody = driver.find_element(By.ID, 'search_resultsRows') # 챠트 리스트
    rows = tbody.find_elements(By.TAG_NAME, 'a')
    AI_df =  pd.DataFrame(columns = ['title','appID','release_date','price','tag','en_tag','developer','publisher','req_min','req_rec','features','lang','evaluation','describe','review','label','images'])
    for y,row in enumerate(tqdm_notebook(rows[:])):
        release_date = None                  # 출시일
        price = None                         # 가격
        evaluation = None                    # 총평가
        evaluation_detail = None             # 세부평가
        title = None                         # 제목
        developer = None                     # 개발
        publisher = None                     # 배급
        single = None                        # 싱글플레이어
        multi = None                         # 멀티플레이어
        online_PvP = None                    # 온라인 PvP
        lan_PvP = None                       # LAN PvP
        shared_Split_Screen_PvP = None       # 스크린 공유 및 분할 PvP
        online_Coop = None                   # 온라인 협동
        lan_Coop = None                      # LAN 협동
        shared_Split_Screen_Coop = None      # 스크린 공유 및 분할 협동
        cross_Platform_Multiplayer = None    # 플롯폼간 멀티플레이어
        lang = None                          # 언어
        req_min_os = None                    # 최저사양 os
        req_min_process = None               # 최저사양 프로세서
        req_min_memory = None                # 최저사양 메모리
        req_min_graphiccard = None           # 최저사양 그래픽
        req_min_storage = None               # 최저사양 저장공간
        req_rec_os = None                    # 권장사양 os
        req_rec_process = None               # 권장사양 프로세서
        req_rec_memory = None                # 권장사양 메모리
        req_rec_graphiccard = None           # 권장사양 그래픽
        req_rec_storage = None               # 권장사양 저장공간
        rev_posi = []
        rev_nega = []
        review = []
        label = []
        describe=[]                           # 게임 설명
        img=None                             # 이미지 저장할때 넣는 변수
        images=[]                             # DataFrame에 저장할 이미지 리스트 
        tag = []
        appID = None
        features_list1 = []
        req_min = []
        req_rec = []
        total_rate=None
        print('---출시일')
        temp = row.find_element(By.XPATH,'//*[@id="search_resultsRows"]/a/div[2]/div[2]')
        if temp == []:
            pass
        else:
            print(temp.text)
            release_date = temp.text

        print('---가격')
        price = row.find_element(By.CLASS_NAME, 'col.search_price_discount_combined.responsive_secondrow').text
        print(price)


        print('---총 평가')
        try:
            review_point = row.find_element(By.CLASS_NAME, 'search_review_summary').get_attribute('data-tooltip-html').split(' ')
            temp_evaluation = review_point[0].split("<br>")
            evaluation = temp_evaluation[0]
            if('Mixed' in evaluation):
                print('---세부 평가')
                percent =  temp_evaluation[1]# 긍정적으로 평가한 사람들의 비율만 추출해서 저장
                amount = review_point[3]
                say = review_point[3] + '명의 사용자 평가했고, 그 중 ' + percent + '가 긍정적으로 평가한 게임입니다.'
                total_rate =  evaluation + "/" +  amount + "/" + percent
                print(say)
                evaluation_detail = say
            check_evaluation = review_point[0] +" "+ review_point[1]
            test_percent = check_evaluation.split('<br>')
            if('Positive' in check_evaluation):
                evaluation = test_percent[0]
                print('---세부 평가')
                percent =  test_percent[1]# 긍정적으로 평가한 사람들의 비율만 추출해서 저장
                amount = review_point[4]
                say = review_point[4] + '명의 사용자 평가했고, 그 중 ' + percent + '가 긍정적으로 평가한 게임입니다.'
                total_rate =  evaluation + "/" +  amount + "/" + percent
                print(say)
                evaluation_detail = say
            elif('Negative' in check_evaluation):
                evaluation = test_percent[0]
                print('---세부 평가')
                percent = test_percent[1] # 긍정적으로 평가한 사람들의 비율만 추출해서 저장
                amount = review_point[4]
                say = review_point[4] + '명의 사용자 평가했고, 그 중 ' + percent + '가 긍정적으로 평가한 게임입니다.'
                total_rate =  evaluation + "/" +  amount + "/" + percent
                print(say)
                evaluation_detail = say
        except:
            evaluation = None
            percent = None
            amount = None
            print(evaluation)
            say = "리뷰를 종합하여 총점을 계산중입니다."
            evaluation_detail = say
            print(say)
        try:
            url = row.get_attribute('href')
            list2= url.split("/")
            list2 = list2[0:len(list2)-1]
            url = '/'.join(list2)

            appID = list2[4]
            print(list2)
            response = requests.get(url)
            response.raise_for_status()
                # Response에서 에러가 발생할 때
                # 프로그램을 중단하도록 할 때는
                # Response 객체의 raise_for_status() 메서드를 호출하여 코드를 불러옴

            if response.status_code == 200:  # 정상
                html = response.text
                soup = BeautifulSoup(html, 'html.parser')
                body = soup.select_one('body')
                tbody = soup.select_one('#tabletGrid > div.page_content_ctn')
            else :
                print(response.status_code)
                exit()
    #===========================================================     
            driver1 = webdriver.Chrome()
            driver1.get(url)
            time.sleep(1)

            if 'app' in body['class']:   # 패키지가 아닌 상품
                print('---제목\n', tbody.select_one('#appHubAppName').text)
                title = tbody.select_one('#appHubAppName').text

                rightcol = tbody.select_one('div.rightcol.game_meta_data')
                genr = rightcol.select_one('#genresAndManufacturer')

                print('---이미지\n')
                print('1')
                try:
                    img=tbody.select_one('.responsive_page_header_img img')
                    try:
                        #                               -----------해당부분에 경로와 이미지 이름을 지정해주세요 
                        rep_title=rep_char(title)
                        urllib.request.urlretrieve(img['src'],'D:\\Python_Example\\steam_images\\'+ rep_title + '_img.jpg')
                        print('이미지 저장 완료')
                        images=rep_title+'_img.jpg'
                    except:
                        print('이미지를 저장하지 못했습니다.')
                        pass
                except:
                    print('이미지가 없습니다.')
                    pass

                print('---설명\n')
                try:
                    desc=tbody.select_one('.game_description_snippet').text
                    print(desc)
                    describe=translator.translate(desc,dest="ko").text
                except:
                    describe='설명 없음'
                    pass

                print(describe)
                # 장르를 기반으로 한 추천에서 태그를 기반으로 하는 추천으로 변경
    #             if genr.find('span') == None:
    #                 genre = 'None'
    #                 pass
    #             else:
    #                 print('---장르\n', genr.find('span').text)   # 장르
    #                 genre = genr.find('span').text
                """
                    개발자
                    배급사
                """

                print('---개발,배급')
                try:
                    developer = genr.select('.dev_row')[0].text
                    developer = re.sub(r'\n',' ',developer)
                    print(developer)
                except:
                    developer = "개발자 정보가 존재하지 않습니다."
                    print(developer)
                try:
                    publisher = genr.select('.dev_row')[1].text
                    publisher = re.sub(r'\n',' ',publisher)
                    print(publisher)
                except:
                    publisher = "배급사 정보가 존재하지 않습니다."
                    print(publisher)

                print('---특징')
                if rightcol.select_one('div.game_area_features_list_ctn') == None:
                    Single = None
                    Multi = None
                    Online_PvP = None
                    LAN_PvP = None
                    Shared_Split_Screen_PvP = None
                    Online_Coop = None
                    LAN_Coop = None
                    Shared_Split_Screen_Coop = None
                    Cross_Platform_Multiplayer = None
                    pass

                else:
                    features_list = rightcol.select_one('div.game_area_features_list_ctn').text
                if 'Single' in features_list:
                    single = '싱글플레이어'
                    features_list1.append(single)
                    print(single)

                if 'Multi' in features_list:
                    Multi = '멀티플레이어'
                    features_list1.append(Multi)
                    print(Multi)

                if 'Online PvP' in features_list:
                    Online_PvP = '온라인 PvP'
                    features_list1.append(Online_PvP)
                    print(Online_PvP)

                if 'LAN PvP' in features_list:
                    LAN_PvP = 'LAN PvP'
                    features_list1.append(LAN_PvP)
                    print(LAN_PvP)

                if 'Shared/Split Screen PvP' in features_list:
                    Shared_Split_Screen_PvP = '스크린 공유 및 분할 PvP'
                    features_list1.append(Shared_Split_Screen_PvP)
                    print(Shared_Split_Screen_PvP)

                if 'Online Co-op' in features_list:
                    Online_Coop = '온라인 협동'
                    features_list1.append(Online_Coop)
                    print(Online_Coop)

                if 'LAN Co-op' in features_list:
                    LAN_Coop = 'LAN 협동'
                    features_list1.append(LAN_Coop)
                    print(LAN_Coop)

                if 'Shared/Split Screen Co-op' in features_list:
                    Shared_Split_Screen_Coop = '스크린 공유 및 분할 협동'
                    features_list1.append(Shared_Split_Screen_Coop)
                    print(Shared_Split_Screen_Coop)

                if 'Cross-Platform Multiplayer' in features_list:
                    Cross_Platform_Multiplayer = '플랫폼간 멀티플레이어'
                    features_list1.append(Cross_Platform_Multiplayer)
                    print(Cross_Platform_Multiplayer)
                features = ",".join(features_list1)
                features = re.sub(r'\n',',',features)
                print('---사양')
                sys_req_tb = tbody.select_one('div.game_area_sys_req')

                req_min_os = None
                req_min_proc = None
                req_min_mem = None
                req_min_grap = None
                req_min_stor = None
                req_rec_os = None
                req_rec_pros = None
                req_rec_mem = None
                req_rec_grap = None
                req_rec_stor = None

                if sys_req_tb == None:
                    pass
                else:
                    sys_req = sys_req_tb.find_all('li')
                    n=int(len(sys_req)/2)
                    print('최소사양')
                    for i in range(n):
                        if 'OS:' in sys_req[i].text:
                            req_min_os = sys_req[i].text.replace('OS: ','')
                            req_min.append(req_min_os)
                            print(req_min_os)
                        if 'Processor:' in sys_req[i].text:
                            req_min_proc = sys_req[i].text.replace('Processor: ','')
                            req_min.append(req_min_proc)
                            print(req_min_proc)
                        if 'Memory:' in sys_req[i].text:
                            req_min_mem = sys_req[i].text.replace('Memory: ','')
                            req_min.append(req_min_mem)
                            print(req_min_mem)
                        if 'Graphics:' in sys_req[i].text:
                            req_min_grap = sys_req[i].text.replace('Graphics: ','')
                            req_min.append(req_min_grap)
                            print(req_min_grap)
                        if 'Storage:' in sys_req[i].text:
                            req_min_stor = sys_req[i].text.replace('Storage: ','')
                            req_min.append(req_min_stor)
                            print(req_min_stor)
                    req_min = ",".join(req_min)
                    print('권장사양')
                    for i in range(n,2*n):
                        if 'OS:' in sys_req[i].text:
                            req_rec_os = sys_req[i].text.replace('OS: ','')
                            req_rec.append(req_rec_os)
                            print(req_rec_os)
                        if 'Processor:' in sys_req[i].text:
                            req_rec_proc = sys_req[i].text.replace('Processor: ','')
                            req_rec.append(req_rec_proc)
                            print(req_rec_proc)
                        if 'Memory:' in sys_req[i].text:
                            req_rec_mem = sys_req[i].text.replace('Memory: ','')
                            req_rec.append(req_rec_mem)
                            print(req_rec_mem)
                        if 'Graphics:' in sys_req[i].text:
                            req_rec_grap = sys_req[i].text.replace('Graphics: ','')
                            req_rec.append(req_rec_grap)
                            print(req_rec_grap)
                        if 'Storage:' in sys_req[i].text:
                            req_rec_stor = sys_req[i].text.replace('Storage: ','')
                            req_rec.append(req_rec_stor)
                            print(req_rec_stor)
                    req_rec = ",".join(req_rec)

                print('---언어')
                if rightcol.select_one('table.game_language_options') == None:
                    lang = None
                    print("여기")
                    pass
                else:
                    lang = ''
                    lang_list = []
                    languages = rightcol.select_one('table.game_language_options').find_all('tr')
                   
                    for language in languages[1:]:
                        lang_list.append(language.find_all('td')[0].text.strip())
                    if "Korean" in lang_list:
                        lang = 'Korean : Y'
                    else:
                        lang = 'Korean : N'
                    if "English" in lang_list:
                        lang += ' English : Y'
                    else:
                        lang = 'English : N'
                    print("==============================================",lang)
#                         지원하는 언어 전체 크롤링이 아닌 한국어, 영어 지원 여부만 저장하는 것으로 변경
#                         차후에 사용될 수 있기 떄문에 소스는 주석처리
#                         print(language.find_all('td')[0].text.replace('\r','').replace('\n','').replace('\t',''))
#                         lang += language.find_all('td')[0].text.replace('\r','').replace('\n',',').replace('\t','')
                print('======================================================================================\n')
                #리뷰===================================================================================================

                review_driver = webdriver.Chrome()
                review_driver.get(url)
                time.sleep(1)

                try:
                    abc = review_driver.find_element(By.ID, 'ageYear')
                    abc.find_elements(By.TAG_NAME, 'option')[40].click()
                    print(review_driver.find_element(By.XPATH, '//*[@id="view_product_page_btn"]').text)
                    review_driver.find_element(By.XPATH, '//*[@id="view_product_page_btn"]').click()
                    time.sleep(2)
                except:
                    pass
                print("---태그\n")
                try:
                    f_taglist = review_driver.find_element(By.CSS_SELECTOR,'#glanceCtnResponsiveRight > div.glance_tags_ctn.popular_tags_ctn > div.glance_tags.popular_tags > div')
                    f_taglist.click()
                    print("클릭 성공")
                    time.sleep(1)
                    en_tag = review_driver.find_element(By.CLASS_NAME,'app_tags.popular_tags').text
                    en_tag = re.sub(r'\n',',',en_tag)
                    review_driver.find_element(By.CSS_SELECTOR,"body > div.newmodal.app_tag_modal_frame > div.newmodal_header_border > div > div.newmodal_close").click()
                    time.sleep(1)
                    try:
                        lang_list = review_driver.find_element(By.CSS_SELECTOR,"#language_pulldown")
                        lang_list.click()
                        lang_ko = review_driver.find_element(By.CSS_SELECTOR,"#language_dropdown > div > a:nth-child(4)")
                        lang_ko.click()
                        time.sleep(1)
                        print("응 성공")
                    except Exception as e1:
                        print(e1,"실패 ㅠㅠㅠㅠㅠㅠ")
                    s_taglist = review_driver.find_element(By.CSS_SELECTOR,'#glanceCtnResponsiveRight > div.glance_tags_ctn.popular_tags_ctn > div.glance_tags.popular_tags > div')
                    s_taglist.click()
                    print("클릭 성공")
                    time.sleep(1)
                    tag = review_driver.find_element(By.CLASS_NAME,'app_tags.popular_tags').text
#                     tag = translator.translate(en_tag,dest="ko").text
                    tag = re.sub(r'\n',',',tag)
                    print("==========================================영어 태그",en_tag)
                    print("===========================================한국어 태그",tag)
                except Exception as e1:
                    print("=============================",e1)
                    tag.append("태그 없음")
                    print(tag)
                print('---리뷰')  
                rewview_last_height = review_driver.execute_script("return document.body.scrollHeight")

                while True:
                    print("=====================확인용")
                    # 끝까지 스크롤 다운
                    review_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    # 1초 대기
                    time.sleep(SCROLL_PAUSE_SEC)

                    # 스크롤 다운 후 스크롤 높이 다시 가져옴
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height

                #리뷰 전체보기
                try:
                    all_review=review_driver.find_element(By.XPATH,'//*[@id="ViewAllReviewssummary"]/a')
                    all_review.send_keys(Keys.ENTER)
                    time.sleep(2)
                    try:
                        #자극적인 리뷰 페이지임을 인지하는지 확인하는 페이지로 들어올경우
                        sexual_content_check=review_driver.find_element(By.XPATH,
                            '//*[@id="responsive_page_template_content"]/div/div/div[2]/button[1]')
                        sexual_content_check.send_keys(Keys.ENTER)
                        time.sleep(2)
                    except Exception as e1:
                        print("에러 확인",e1)
                        pass

                    #한국어 리뷰만 보기
                    language=review_driver.find_element(By.ID,'filterlanguage_activeday')
                    language.click()
                    time.sleep(1)
                    korean=review_driver.find_element(By.ID,'filterlanguage_option_5')
                    korean.click()
                    time.sleep(1)

                    filterrating=review_driver.find_element(By.ID,'filterrating')
                    filterrating.click()
                    time.sleep(1)

                    #긍정적인 평가 필터링
                    positive=review_driver.find_element(By.ID,'filterrating_option_1')
                    positive.click()
                    time.sleep(1)

                    review_texts=review_driver.find_elements(By.CLASS_NAME,'apphub_CardTextContent')
                    if(len(review_texts)>=8):
                        for review_text in review_texts[:8]: # 8개만 0~7리뷰 없다. indexError
                            rev_posi.append(review_text.text[review_text.text.find('\n')+1:]) #날짜 빼기
                            label.append(1)
                    else:# 리뷰가 8개가 안된다.
                        for review_text in review_texts[:len(review_texts)]: # 8개만 0~7리뷰 없다. indexError
                            rev_posi.append(review_text.text[review_text.text.find('\n')+1:]) #날짜 빼기
                            label.append(1)
                        for i in range(8-len(rev_posi)):
                            rev_posi.append(None)
                            label.append(1)
                    time.sleep(1)
                    #부정적인 평가 필터링
                    filterrating=review_driver.find_element(By.ID,'filterrating')
                    filterrating.click()
                    time.sleep(1)
                    negative=review_driver.find_element(By.ID,'filterrating_option_2')
                    negative.click()
                    time.sleep(1)

                    review_texts=review_driver.find_elements(By.CLASS_NAME,'apphub_CardTextContent')
                    if(len(review_texts)>=8):
                        for review_text in review_texts[:8]: # 8개만 0~7리뷰 없다. indexError
                            rev_nega.append(review_text.text[review_text.text.find('\n')+1:]) #날짜 빼기
                            label.append(0)
                    else:# 리뷰가 8개가 안된다.
                        for review_text in review_texts[:len(review_texts)]: # 8개만 0~7리뷰 없다. indexError
                            rev_nega.append(review_text.text[review_text.text.find('\n')+1:]) #날짜 빼기
                            label.append(0)
                        for i in range(8-len(rev_nega)):
                            rev_nega.append(None)
                            label.append(0)
                except Exception as e1:
                    for i in range(8-len(rev_posi)):
                        rev_posi.append(None)
                        label.append(1)
                    for i in range(8-len(rev_nega)):
                        rev_nega.append(None)
                        label.append(0)
                    print("에러 확인",e1)
                    pass

                review_driver.quit()
                #리뷰===================================================================================================
                review = rev_posi+rev_nega
                for i in range(len(review)):
                    data = {
                        'title':[title],
                        'appID':[appID],
                        'price':[price],
                        'evaluation':[total_rate],
                        'review':review[i],
                        'describe':[describe],
                        'label' :label[i],
                        'req_rec':[req_rec],
                        'req_min':[req_min],
                        'lang':[lang],
                        'features':[features],
                        'developer':[developer],
                        'publisher':[publisher],
                        'tag':[tag],
                        'en_tag':[en_tag],
                        'images':[images],
                        'release_date':[release_date]
                    }

                    temp_df = pd.DataFrame(data)
                    AI_df = pd.concat([AI_df,temp_df], ignore_index=True)
                print(AI_df)

                print('======================================================================================\n')
                driver1.close()
            else:                         # 패키지인 상품
                print('---제목\n', tbody.select_one('h2.pageheader').text)
                title = tbody.select_one('h2.pageheader').text

                rightcol = tbody.select_one('div.rightcol.game_meta_data')

                print('---이미지\n')
                print('2')
                try:
                    img=tbody.select_one('.responsive_page_header_img img')
                    try:
                        #                               -----------해당부분에 경로와 이미지 이름을 지정해주세요 
                        rep_title=rep_char(title)
                        urllib.request.urlretrieve(img['src'],'D:\\Python_Example\\steam_images\\'+ rep_title + '_img.jpg')
                        print('이미지 저장 완료')
                        images=rep_title+'_img.jpg'
                    except:
                        print('이미지를 저장하지 못했습니다.')
                        pass
                except:
                    print('이미지가 없습니다.')
                    pass

                print('---설명\n')
                try:
                    desc=tbody.select_one('.game_description_snippet').text
                    print(desc)
                    describe=translator.translate(desc,dest="ko").text
                except:
                    describe='설명 없음'
                    pass
                print(describe)
                print('---장르\n', genr[0].text)   # 장르

                """
                    개발자
                    배급사
                """

                print('---개발,배급')
                try:
                    developer = genr.select('.dev_row')[0].text
                    developer = re.sub(r'\n',' ',developer)
                    print(developer)
                except:
                    developer = "개발자 정보가 존재하지 않습니다."
                    print(developer)
                try:
                    publisher = genr.select('.dev_row')[1].text
                    publisher = re.sub(r'\n',' ',publisher)
                    print(publisher)
                except:
                    publisher = "배급사 정보가 존재하지 않습니다."
                    print(publisher)

                print('---특징')
                features_list = rightcol.select_one('div.details_block.details_specs_ctn')
                if features_list == None:
                    features_list = rightcol.select('div.details_block')[1].text
                    Single = None
                    Multi = None
                    Online_PvP = None
                    LAN_PvP = None
                    Shared_Split_Screen_PvP = None
                    Online_Coop = None
                    LAN_Coop = None
                    Shared_Split_Screen_Coop = None
                    Cross_Platform_Multiplayer = None

                else:
                    features_list = rightcol.select_one('div.game_area_features_list_ctn').text
                if 'Single' in features_list:
                    single = '싱글플레이어'
                    features_list1.append(single)
                    print(single)

                if 'Multi' in features_list:
                    Multi = '멀티플레이어'
                    features_list1.append(Multi)
                    print(Multi)

                if 'Online PvP' in features_list:
                    Online_PvP = '온라인 PvP'
                    features_list1.append(Online_PvP)
                    print(Online_PvP)

                if 'LAN PvP' in features_list:
                    LAN_PvP = 'LAN PvP'
                    features_list1.append(LAN_PvP)
                    print(LAN_PvP)

                if 'Shared/Split Screen PvP' in features_list:
                    Shared_Split_Screen_PvP = '스크린 공유 및 분할 PvP'
                    features_list1.append(Shared_Split_Screen_PvP)
                    print(Shared_Split_Screen_PvP)

                if 'Online Co-op' in features_list:
                    Online_Coop = '온라인 협동'
                    features_list1.append(Online_Coop)
                    print(Online_Coop)

                if 'LAN Co-op' in features_list:
                    LAN_Coop = 'LAN 협동'
                    features_list1.append(LAN_Coop)
                    print(LAN_Coop)

                if 'Shared/Split Screen Co-op' in features_list:
                    Shared_Split_Screen_Coop = '스크린 공유 및 분할 협동'
                    features_list1.append(Shared_Split_Screen_Coop)
                    print(Shared_Split_Screen_Coop)

                if 'Cross-Platform Multiplayer' in features_list:
                    Cross_Platform_Multiplayer = '플랫폼간 멀티플레이어'
                    features_list1.append(Cross_Platform_Multiplayer)
                    print(Cross_Platform_Multiplayer)
                features = ",".join(features_list1)
                features = re.sub(r'\n',',',features)
                print('---언어')
                language = rightcol.select_one('span.language_list')
                print(language.text.replace('Listed languages may not be available for all games in the package. View the individual games for more details.','').replace('LANGUAGES: ','').replace('\n','').replace('\t','').replace('Languages: ',''))
               #리뷰===================================================================================================

                review_driver = webdriver.Chrome()
                review_driver.get(url)
                time.sleep(1)
                print("---태그\n")
                try:
                    taglist = review_driver.find_element(By.CSS_SELECTOR,'#glanceCtnResponsiveRight > div.glance_tags_ctn.popular_tags_ctn > div.glance_tags.popular_tags > div')
                    print(taglist)
                    taglist.click()
                    print("클릭 성공")
                    time.sleep(1)
                    en_tag = review_driver.find_element(By.CLASS_NAME,'app_tags.popular_tags').text
                    tag = translator.translate(en_tag,dest="ko").text
                    tag = re.sub(r'\n',',',tag)
                    print(tag)
                except Exception as e1:
                    print("=============================",e1)
                    tag.append("태그 없음")
                    print(tag)
                print('---리뷰')
                try:
                    abc = review_driver.find_element(By.ID, 'ageYear')
                    abc.find_elements(By.TAG_NAME, 'option')[40].click()
                    time.sleep(1)
                    print(review_driver.find_element(By.XPATH, '//*[@id="view_product_page_btn"]').text)
                    review_driver.find_element(By.XPATH, '//*[@id="view_product_page_btn"]').click()
                    time.sleep(2)
                except:
                    pass

                rewview_last_height = review_driver.execute_script("return document.body.scrollHeight")

                while True:
                    print("=====================확인용")
                    # 끝까지 스크롤 다운
                    review_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    # 1초 대기
                    time.sleep(SCROLL_PAUSE_SEC)

                    # 스크롤 다운 후 스크롤 높이 다시 가져옴
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height

                #리뷰 전체보기
                try:
                    all_review=review_driver.find_element(By.XPATH,'//*[@id="ViewAllReviewssummary"]/a')
                    all_review.send_keys(Keys.ENTER)
                    time.sleep(2)
                    try:
                        #자극적인 리뷰 페이지임을 인지하는지 확인하는 페이지로 들어올경우
                        sexual_content_check=review_driver.find_element(By.XPATH,
                            '//*[@id="responsive_page_template_content"]/div/div/div[2]/button[1]')
                        sexual_content_check.send_keys(Keys.ENTER)
                        time.sleep(2)
                    except Exception as e1:
                        print("에러 확인",e1)
                        pass
                    time.sleep(1)
                    #한국어 리뷰만 보기
                    language=review_driver.find_element(By.ID,'filterlanguage_activeday')
                    language.click()
                    time.sleep(1)
                    korean=review_driver.find_element(By.ID,'filterlanguage_option_5')
                    korean.click()
                    time.sleep(1)

                    filterrating=review_driver.find_element(By.ID,'filterrating')
                    filterrating.click()
                    time.sleep(1)

                    #긍정적인 평가 필터링
                    positive=review_driver.find_element(By.ID,'filterrating_option_1')
                    positive.click()
                    time.sleep(1)

                    review_texts=review_driver.find_elements(By.CLASS_NAME,'apphub_CardTextContent')
                    if(len(review_texts)>=8):
                        for review_text in review_texts[:8]: # 8개만 0~7리뷰 없다. indexError
                            rev_posi.append(review_text.text[review_text.text.find('\n')+1:]) #날짜 빼기
                            label.append(1)
                    else:# 리뷰가 8개가 안된다.
                        for review_text in review_texts[:len(review_texts)]: # 8개만 0~7리뷰 없다. indexError
                            rev_posi.append(review_text.text[review_text.text.find('\n')+1:]) #날짜 빼기
                            label.append(1)
                        for i in range(8-len(rev_posi)):
                            rev_posi.append(None)
                            label.append(1)

                    time.sleep(2)
                    #부정적인 평가 필터링
                    filterrating=review_driver.find_element(By.ID,'filterrating')
                    filterrating.click()
                    time.sleep(1)
                    negative=review_driver.find_element(By.ID,'filterrating_option_2')
                    negative.click()
                    time.sleep(1)

                    review_texts=review_driver.find_elements(By.CLASS_NAME,'apphub_CardTextContent')
                    if(len(review_texts)>=8):
                        for review_text in review_texts[:8]: # 8개만 0~7리뷰 없다. indexError
                            rev_nega.append(review_text.text[review_text.text.find('\n')+1:]) #날짜 빼기
                            label.append(0)
                    else:# 리뷰가 8개가 안된다.
                        for review_text in review_texts[:len(review_texts)]: # 8개만 0~7리뷰 없다. indexError
                            rev_nega.append(review_text.text[review_text.text.find('\n')+1:]) #날짜 빼기
                            label.append(0)
                        for i in range(8-len(rev_nega)):
                            rev_nega.append(None)
                            label.append(0)
                except Exception as e1:
                    for i in range(8-len(rev_posi)):
                        rev_posi.append(None)
                        label.append(1)
                    for i in range(8-len(rev_nega)):
                        rev_nega.append(None)
                        label.append(0)
                    print("에러 확인",e1)
                    pass

                review_driver.quit()
                #리뷰===================================================================================================
                review = rev_posi+ rev_nega

                for i in range(len(review)):
                    data = {
                        'title':[title],
                        'appID':[appID],
                        'price':[price],
                        'evaluation':[total_rate],
                        'genre':[genre],
                        'review':review[i],
                        'describe':[describe],
                        'label' :label[i],
                        'req_rec':[req_rec],
                        'req_min':[req_min],
                        'lang':[lang],
                        'features':[features],
                        'developer':[developer],
                        'publisher':[publisher],
                        'tag':[tag],
                        'en_tag': [en_tag],
                        'images':[images],
                        'release_date':[release_date]
                    }
                    temp_df = pd.DataFrame(data)
                    AI_df = pd.concat([AI_df,temp_df], ignore_index=True)
                print(AI_df)
                driver1.close()
                print('======================================================================================\n')

            AI_df.to_csv("test.csv")        
        except Exception as e1:
            print(e1,'error')

In [17]:
def pretreatment():
    AI_df_fin = pd.read_csv("test.csv")
    # AI 학습용 리뷰 전처리 입니다.
    list_ = list(AI_df_fin['review'])
    list_2 = list(AI_df_fin['tag'])
    for i in tqdm_notebook(range(len(AI_df_fin))):
        try:
            list_[i] = re.sub(r"[^가-힣|^' ']", "", list_[i])
            list_2[i] = re.sub(r'\n',' ',list_2[i])
            AI_df_fin.loc[[i],['review']] = list_[i]
            AI_df_fin.loc[[i],['tag']] = list_2[i]
            print(AI_df_fin.loc[[i],['review']])
        except:
            print(list_[i])
            AI_df_fin.loc[[i],['review']] = list_[i]
            pass
    # AI 학습용 데이터 null값 처리 코드
    AI_df_fin = AI_df_fin.dropna(subset = ['review'],axis=0)
    AI_df_fin = AI_df_fin.reset_index(drop = True)
    AI_df_fin
    # 전처리 끝난 데이터 파일로 저장
    AI_df_fin.to_csv("스팀 크롤링 데이터(2024-03-12).csv")

In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook
from googletrans import Translator
import urllib
import re
def main():
    driver = webonload()
    crawling(driver)  
    pretreatment()

In [19]:
main()

C:\Users\user\AppData\Local\Temp\ipykernel_18488\2555187349.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for y,row in enumerate(tqdm_notebook(rows[:])):


  0%|          | 0/4467 [00:00<?, ?it/s]

---출시일
8 Feb, 2024
---가격
₩ 44,800
---총 평가
---세부 평가
215,121명의 사용자 평가했고, 그 중 82%가 긍정적으로 평가한 게임입니다.
['https:', '', 'store.steampowered.com', 'app', '553850', 'HELLDIVERS_2']
---제목
 HELLDIVERS™ 2
---이미지

1
이미지를 저장하지 못했습니다.
---설명


								The Galaxy’s Last Line of Offence. Enlist in the Helldivers and join the fight for freedom across a hostile galaxy in a fast, frantic, and ferocious third-person shooter.							
갤럭시의 마지막 범죄 라인.Helldivers에 입대하고 빠르고 열광적이며 사나운 3 인칭 사수로 적대적인 은하계를 가로 지르는 자유를위한 싸움에 참여하십시오.
---개발,배급
 Developer: Arrowhead Game Studios 
 Publisher: PlayStation PC LLC 
---특징
온라인 협동
---사양
최소사양
Windows 10
Intel Core i7-4790K or AMD Ryzen 5 1500X
8 GB RAM
NVIDIA GeForce GTX 1050 Ti or AMD Radeon RX 470
100 GB available space
권장사양
Windows 10
Intel Core i7-9700K or AMD Ryzen 7 3700X
16 GB RAM
NVIDIA GeForce RTX 2060 or AMD Radeon RX 6600XT
100 GB available space
---언어
============================================== Korean : Y English : Y

View Page
---태그

클릭 성공
응 성공
클릭 성공
===============

['https:', '', 'store.steampowered.com', 'app', '578080', 'PUBG_BATTLEGROUNDS']
---제목
 PUBG: BATTLEGROUNDS
---이미지

1
이미지를 저장하지 못했습니다.
---설명


								Play PUBG: BATTLEGROUNDS for free. Land on strategic locations, loot weapons and supplies, and survive to become the last team standing across various, diverse Battlegrounds. Squad up and join the Battlegrounds for the original Battle Royale experience that only PUBG: BATTLEGROUNDS can offer.							
PUBG : 무료로 배경.전략적 위치, 전리품 무기 및 용품에 착륙하여 살아 남기 위해 다양한 전장을 가로 질러 서있는 마지막 팀이 되십시오.PUBG : Battlegrounds 만 제공 할 수있는 최초의 Battle Royale Experience를 위해 전장을 대고 전장에 가입하십시오.
---개발,배급
 Developer: KRAFTON, Inc. 
 Publisher: KRAFTON, Inc. 
---특징
온라인 PvP
---사양
최소사양
64-bit Windows 10
Intel Core i5-4430 / AMD FX-6300
8 GB RAM
NVIDIA GeForce GTX 960 2GB / AMD Radeon R7 370 2GB
40 GB available space
권장사양
64-bit Windows 10
Intel Core i5-6600K / AMD Ryzen 5 1600
16 GB RAM
NVIDIA GeForce GTX 1060 3GB / AMD Radeon RX 580 4GB
50 GB available space
---언어
=============

---제목
 Dragon's Dogma 2
---이미지

1
이미지를 저장하지 못했습니다.
---설명


								Dragon’s Dogma 2 is a single player, narrative driven action-RPG that challenges the players to choose their own experience – from the appearance of their Arisen, their vocation, their party, how to approach different situations and more - in a truly immersive fantasy world.							
Dragon 's Dogma 2는 싱글 플레이어, 이야기 중심의 액션 -RPG로, 플레이어가 자신의 경험을 선택하도록 도전합니다..
---개발,배급
 Developer: CAPCOM Co., Ltd. 
 Publisher: CAPCOM Co., Ltd. 
---특징
싱글플레이어
---사양
최소사양
Windows 10 (64 bit)/Windows 11 (64 bit)
Intel Core i5 10600 /  AMD Ryzen 5 3600
16 GB RAM
NVIDIA GeForce GTX 1070 / AMD Radeon RX 5500 XT with 8GB VRAM
권장사양
Windows 10 (64 bit)/Windows 11 (64 bit)
Intel Core i7-10700 /  AMD Ryzen 5 3600X
16 GB RAM
NVIDIA GeForce RTX 2080 / AMD Radeon RX 6700
---언어
============================================== Korean : Y English : Y

View Page
---태그

클릭 성공
응 성공
클릭 성공
==========================================영어 태그 RPG,Action,Open World,Action RP

...

---제목
 Yu-Gi-Oh! Master Duel
---이미지

1
이미지를 저장하지 못했습니다.
---설명


								Yu-Gi-Oh! MASTER DUEL is the ultimate free-to-play cross-platform strategy card game with fast-paced Duels, stunning HD graphics and a new, dynamic soundtrack! Get ready to challenge Duelists around the world!							
유기 오!Mas전 세계의 듀얼리스트에게 도전 할 준비를하십시오!
---개발,배급
 Developer: KONAMI 
 Publisher: KONAMI 
---특징
싱글플레이어
멀티플레이어
온라인 PvP
플랫폼간 멀티플레이어
---사양
최소사양
Windows 10 Home (64bit) latest revision
Intel Core i5-7300U
8 GB RAM
Intel(R) HD Graphics 620
16 GB available space
권장사양
Windows 10 Home (64bit) latest revision
Intel Core i5-9400F
8 GB RAM
NVIDIA GeForce GTX 1650
16 GB available space
---언어
============================================== Korean : Y English : Y

---태그

클릭 성공
응 성공
클릭 성공
==========================================영어 태그 Card Game,Free to Play,Strategy,Trading Card Game,Multiplayer,PvP,Card Battler,Anime,Simulation,Turn-Based Tactics,Tactical,Survival Horror,Singleplayer,eSports,Fantasy,Dragons,PvE,3D,2D,Gre

---제목
 Apex Legends™
---이미지

1
이미지를 저장하지 못했습니다.
---설명


								Apex Legends is the award-winning, free-to-play Hero Shooter from Respawn Entertainment. Master an ever-growing roster of legendary characters with powerful abilities, and experience strategic squad play and innovative gameplay in the next evolution of Hero Shooter and Battle Royale.							
Apex Legends는 Respawn Entertainment의 수상 경력에 빛나는 무료 플레이 영웅 슈팅 게임입니다.강력한 능력을 가진 전설적인 캐릭터로 구성된 명단을 습득하고 다음의 영웅 슈팅 게임과 배틀 로얄의 진화에서 전략적 분대 놀이와 혁신적인 게임 플레이를 경험하십시오.
---개발,배급
 Developer: Respawn Entertainment 
 Publisher: Electronic Arts 
---특징
온라인 PvP
온라인 협동
---사양
최소사양
64-bit Windows 10
AMD FX 4350 or Equivalent, Intel Core i3 6300 or Equivalent
6 GB RAM
AMD Radeon™ HD 7730, NVIDIA GeForce® GT 640
75 GB available space
권장사양
64-bit Windows 10
Ryzen 5 CPU or Equivalent
8 GB RAM
AMD Radeon™ R9 290, NVIDIA GeForce® GTX 970
75 GB available space
---언어
============================================== Korean : Y English : Y

---태그

클릭 성공
응 성공
클릭 성공
==

Exception ignored in: <function Service.__del__ at 0x000001E4B1E2FCE0>
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "C:\Users\user\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\user\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Users\user\anaconda3\Lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\urllib\request.py", line 536, in _open
    result = self._call_chain(self.handle_open, prot

             title    appID release_date     price  \
0    HELLDIVERS™ 2   553850  8 Feb, 2024  ₩ 44,800   
1    HELLDIVERS™ 2   553850  8 Feb, 2024  ₩ 44,800   
2    HELLDIVERS™ 2   553850  8 Feb, 2024  ₩ 44,800   
3    HELLDIVERS™ 2   553850  8 Feb, 2024  ₩ 44,800   
4    HELLDIVERS™ 2   553850  8 Feb, 2024  ₩ 44,800   
..             ...      ...          ...       ...   
139  Apex Legends™  1172470  8 Feb, 2024      Free   
140  Apex Legends™  1172470  8 Feb, 2024      Free   
141  Apex Legends™  1172470  8 Feb, 2024      Free   
142  Apex Legends™  1172470  8 Feb, 2024      Free   
143  Apex Legends™  1172470  8 Feb, 2024      Free   

                                                   tag  \
0    액션,온라인 협동,멀티플레이어,3인칭 슈팅,슈팅,협동,PvE,3인칭,우주,공상과학,...   
1    액션,온라인 협동,멀티플레이어,3인칭 슈팅,슈팅,협동,PvE,3인칭,우주,공상과학,...   
2    액션,온라인 협동,멀티플레이어,3인칭 슈팅,슈팅,협동,PvE,3인칭,우주,공상과학,...   
3    액션,온라인 협동,멀티플레이어,3인칭 슈팅,슈팅,협동,PvE,3인칭,우주,공상과학,...   
4    액션,온라인 협동,멀티플레이어,3인칭 슈팅,슈팅,협동,PvE,3인칭,우주,공상과학,...   
.. 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.112)
Stacktrace:
	GetHandleVerifier [0x00007FF7F957AD32+56930]
	(No symbol) [0x00007FF7F94EF632]
	(No symbol) [0x00007FF7F93A42E5]
	(No symbol) [0x00007FF7F9381D4C]
	(No symbol) [0x00007FF7F94123F7]
	(No symbol) [0x00007FF7F9427891]
	(No symbol) [0x00007FF7F940BA43]
	(No symbol) [0x00007FF7F93DD438]
	(No symbol) [0x00007FF7F93DE4D1]
	GetHandleVerifier [0x00007FF7F98F6ABD+3709933]
	GetHandleVerifier [0x00007FF7F994FFFD+4075821]
	GetHandleVerifier [0x00007FF7F994818F+4043455]
	GetHandleVerifier [0x00007FF7F9619766+706710]
	(No symbol) [0x00007FF7F94FB90F]
	(No symbol) [0x00007FF7F94F6AF4]
	(No symbol) [0x00007FF7F94F6C4C]
	(No symbol) [0x00007FF7F94E6904]
	BaseThreadInitThunk [0x00007FFBE6C7257D+29]
	RtlUserThreadStart [0x00007FFBE7A2AA58+40]


In [20]:
df = pd.read_csv("스팀 크롤링 데이터(2024-03-12).csv")
translator = Translator()
tag_df_ko = list(df['tag'])
tag_df_en = list(df['en_tag'])
tag_fin = pd.DataFrame()
list_tag_fin =[]
tag_df_tr_list = []
for i in range(len(tag_df_ko)):

    tag_df_ko_list = tag_df_ko[i].split(",")
    tag_df_en_list = tag_df_en[i].split(",")
    for j in range(len(tag_df_ko_list)):
        if tag_df_ko_list[j] == '태그 없음' or tag_df_ko_list[j] == ']' or tag_df_ko_list[j] == '[' or tag_df_ko_list[j] == '[태그 없음]':
            pass
        elif tag_df_ko_list[j] not in list_tag_fin:
            tag_df_tr_list.append(translator.translate(tag_df_ko_list[j],dest="ko").text)
            list_tag_fin.append(tag_df_ko_list[j])
            tag_df_ko_list[j]
            temp_tag_fin = pd.DataFrame({
                'tag':[tag_df_ko_list[j]],
                'en_tag':[tag_df_en_list[j]],
                'tr_tag':[tag_df_tr_list[j]]
            })
            tag_fin = pd.concat([tag_fin,temp_tag_fin],ignore_index = True)
tag_fin.to_excel('tag_df.xlsx',index=False)

FileNotFoundError: [Errno 2] No such file or directory: '스팀 크롤링 데이터(2024-03-12).csv'

def tag_controller:
    df = pd.read_csv("스팀 크롤링 데이터(2024-01-09).csv")
    tag_df = df[['tag','en_tag']]
    list_tag = list(tag_df['tag'])
    list_tag_en = list(tag_df['en_tag'])
    set_tag = set(list_tag)
    list_tag = list(set_tag)
    list_tag_fin =[]
    for i in range(len(list_tag)):
        temp_list = list_tag[i].split(",")
        for j in range(len(temp_list)):
            if temp_list[j] == '태그 없음' or temp_list[j] == ']' or temp_list[j] == '[' or temp_list[j] == '[태그 없음]':
                pass
            elif temp_list[j] not in list_tag_fin:
                print(temp_list[j])
                list_tag_fin.append(temp_list[j])
            else:
                print(f"{temp_list[j]}는 이미 존재한다")
                pass
    temp_df = pd.DataFrame({'tag':list_tag_fin})
    print(temp_df)
    temp_df.to_csv('태그 중복 제거.csv')